<a href="https://colab.research.google.com/github/Santiagoxy/Laboratorios/blob/main/Car_Recommended_Engine_v1editado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors

In [35]:
#Librerias
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error as MAE,r2_score

import lightgbm as lgb

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import cross_val_score, KFold
import xgboost as xgb
from sklearn.metrics import classification_report

In [36]:
url = "/content/drive/MyDrive/Big Data/usedCarsCol_logan_200624.csv"
cols = ["car_model", "price", "year_model", "kms", "color",
        "fueltype"]
data = pd.read_csv(url, names=cols)


In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
print(data.shape)
data.head()

(433, 6)


,car_model,price,year_model,kms,color,fueltype
0,car_model,price,year_model,kms,color,fueltype
1,Renault Logan Expression mecánico,$34.500.000,2018,93.500,Plateado,Gasolina
2,Renault Logan AUTHENTIQUE 1600CC MT AA,$30.500.000,2016,75.200,Plateado,Gasolina
3,Renault Logan Familier 1400cc Mt Aa,$27.000.000,2014,156.450,Gris,Gasolina
4,Renault Logan AUTHENTIQUE 1600CC MT AA,$34.000.000,2016,102.685,Negro,Gasolina


In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433 entries, 0 to 432
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   car_model   433 non-null    object
 1   price       433 non-null    object
 2   year_model  433 non-null    object
 3   kms         433 non-null    object
 4   color       433 non-null    object
 5   fueltype    433 non-null    object
dtypes: object(6)
memory usage: 20.4+ KB


In [40]:
#Valores nulos y en que medidad
data.isnull().sum()

car_model     0
price         0
year_model    0
kms           0
color         0
fueltype      0
dtype: int64

In [41]:
# eliminar los valores nulos de la columna price
data['price'] = data['price'].dropna()

In [42]:
#pd.to_numeric() de pandas para convertir las columnas 'price' y 'horsepower'
data['price'] = pd.to_numeric(data['price'], errors='coerce')


In [43]:
#Recorre todas las columnas del DataFrame data que tienen un tipo de datos object y las convierte en categorías utilizando el método astype('category').
for col in data.select_dtypes(include=['object']).columns:
  data[col] = data[col].astype('category')

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433 entries, 0 to 432
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   car_model   433 non-null    category
 1   price       0 non-null      float64 
 2   year_model  433 non-null    category
 3   kms         433 non-null    category
 4   color       433 non-null    category
 5   fueltype    433 non-null    category
dtypes: category(5), float64(1)
memory usage: 23.3 KB


In [45]:
#Valores nulos y en que medidad
data.isna().sum()

car_model       0
price         433
year_model      0
kms             0
color           0
fueltype        0
dtype: int64

In [46]:
#eliminará todas las filas que contienen al menos un valor nulo y actualizará el DataFrame original.
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   car_model   0 non-null      category
 1   price       0 non-null      float64 
 2   year_model  0 non-null      category
 3   kms         0 non-null      category
 4   color       0 non-null      category
 5   fueltype    0 non-null      category
dtypes: category(5), float64(1)
memory usage: 16.8 KB


In [49]:
# Preprocessing
categorical_features = ['make', 'fuel_type', 'body_style', 'drive_wheels']
numeric_features = ['wheel_base', 'length', 'engine_size', 'horsepower']

selected_columns = categorical_features + numeric_features
new_data = data[selected_columns]

# Display the first few rows of the new dataset
new_data.head()

# Standarization (both categorical and numerical variables)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)])

KeyError: "None of [Index(['make', 'fuel_type', 'body_style', 'drive_wheels', 'wheel_base',\n       'length', 'engine_size', 'horsepower'],\n      dtype='object')] are in the [columns]"

In [ ]:
new_data

In [ ]:
preprocessor

In [ ]:
# Define pipeline for regression model
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])


In [ ]:
model

In [ ]:
# Este bloque de código divide los datos en conjuntos de entrenamiento y prueba para su posterior modelado y evaluación
# Split data: training and testing
# X = new_data.drop('price', axis=1)
X = new_data
y = data['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X.shape

In [ ]:
X_train.head()

In [ ]:
# devolverá las primeras filas de datos en y_train
y_train.head()

In [ ]:
# Training the price prediction model (regression)
model.fit(X_train, y_train)

In [ ]:
# Evaluating the price prediction model (regression)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

### Regresión lineal multivariable
La regresión lineal multivariable es una técnica para predecir una variable (dependiente) basada en múltiples variables predictoras. Modela la relación entre estas variables como una función lineal, donde se buscan coeficientes óptimos que minimicen la diferencia entre los valores observados y los predichos. Es ampliamente utilizada en estadísticas y aprendizaje automático para análisis predictivo y comprensión de relaciones entre variables.

In [ ]:
# accuracy check
mse = mean_squared_error (y_test, y_pred)
rmse = mean_squared_error (y_test, y_pred, squared=False)
mae = MAE (y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % rmse)
print("MAE: %.2f" % mae)
print("R2: %.2f" % r2)

### Light GBM (Gradient Boosting Machine)

Es una biblioteca de aprendizaje automático de código abierto y de alta eficiencia que se utiliza para problemas de regresión, clasificación y ranking. Utiliza el algoritmo de refuerzo de gradiente para construir un modelo predictivo mediante la combinación de múltiples árboles de decisión débiles.

In [ ]:
# Hyperparameters
params = {
    'task': 'train',
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 5,
    'learning_rate': 0.05,
    'metric': {'l2','l1'},
    'header' : 'true',
    'verbose': 0
}

# loading data
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# fitting the model
model2 = lgb.train(params,
                 train_set=lgb_train,
                 valid_sets=lgb_eval)
# Pred
y_pred2 = model2.predict(X_test)

In [ ]:
# accuracy check
mse = mean_squared_error(y_test, y_pred2)
rmse = mean_squared_error(y_test, y_pred2, squared=False)
mae = MAE(y_test, y_pred2)
r2 = r2_score(y_test, y_pred2)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % rmse)
print("MAE: %.2f" % mae)
print("R2: %.2f" % r2)

# Support Vector Regression (SVR)

Es una técnica de aprendizaje automático basada en SVM, que originalmente se desarrolló para tareas de clasificación. En el contexto de regresión, SVR es utilizado para modelar y predecir una variable continua.

In [ ]:
categorical_features = ['make', 'fuel_type', 'body_style', 'drive_wheels']
numeric_features = ['wheel_base', 'length', 'engine_size', 'horsepower']

# Crear transformadores para datos numéricos y categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='drop'
)


# Preparar el pipeline de preprocesamiento y el modelo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svr', SVR(kernel='linear'))
])

X = data.drop('price', axis=1)
y = data['price'].dropna()  # Asegurándose de que no hay valores vacios

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"R-squared: {r2}")

In [ ]:
# Define the function to recommend cars
def recommend_and_predict(car_features_df, data, model):

    # Preprocessing
    categorical_features = ['make', 'fuel_type', 'body_style', 'drive_wheels']
    numeric_features = ['wheel_base', 'length', 'engine_size', 'horsepower']
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(), categorical_features)
        ],
        remainder='drop'
    )

    # Pipeline to transformed data
    pipe = Pipeline(steps=[('preprocessor', preprocessor)])
    transformed_data = pipe.fit_transform(data.drop(['price'], axis=1))
    transformed_query = pipe.transform(car_features_df)

    # Price prediction
    predicted_price = model.predict(car_features_df)
    print(f"Predicted Price: {predicted_price[0]}")

    # Applying K-Nearest Neighbors for searching similar cars
    n_neighbors = 5
    nn = NearestNeighbors(n_neighbors=n_neighbors)
    nn.fit(transformed_data)

    # Finding the nearest neighbors for the input 'car_features_df'
    distances, indices = nn.kneighbors(transformed_query)

    # Obtaining similar cars
    similar_cars = data.iloc[indices[0]]
    return similar_cars

In [ ]:
new_data_y = new_data
new_data_y['price'] = data['price']
new_data_y.head()

In [ ]:
# Example use of the function (ensure your DataFrame and model are correctly defined)
test_data_recommended = pd.DataFrame([{
    'make': 'audi', 'fuel_type': 'gas', 'body_style': 'sedan', 'drive_wheels': '4wd',
    'wheel_base': 80, 'length': 180, 'engine_size': 120, 'horsepower': 100}])

similar_cars = recommend_and_predict(test_data_recommended, new_data_y, model)
print("Recommended Similar Cars:")
print(similar_cars)

#### **Conclusiones**

